## Preprocesando las bases de la ENAHO
Limpiare y modificare los modulos de la ENAHO anual del 2019 con la finalidad de contar con las principales variables que se usaran para la clasificacion

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

Se necesitara la sumaria para extraer la variable de pobreza, el modulo 2 para caracteristicas del hogar, modulo 3 para datos relacionados a la educacion y el modulo 5 para las variables de empleo e ingresos

In [2]:
# Sumaria
set1 = pd.read_stata("./Bases de datos/sumaria-2019.dta",
                     columns=["mes", "ubigeo", "conglome", "vivienda", "ingmo1hd", "pobreza"])

# Modulo 2: Características de los Miembros del Hogar
set2 = pd.read_stata("./Bases de datos/enaho01-2019-200.dta",
                    columns=["mes", "ubigeo", "conglome", "vivienda", "estrato", "codperso", "p208a", "p207", "p209"])

# Modulo 3: Educacion
set3 = pd.read_stata("./Bases de datos/enaho01a-2019-300.dta",
                    columns=["mes", "ubigeo", "conglome", "vivienda", "codperso", "p301a", "p301d"])

# Modulo 5: Empleo e Ingresos
set4 = pd.read_stata("./Bases de datos/enaho01a-2019-500.dta",
                    columns=["mes", "ubigeo", "conglome", "vivienda", "codperso", "p512a", "p514"])

Finalmente, vamos a juntar las bases de datos mediante las variables de `ubigeo`, `conglome`, `vivienda`, `codperso`. Como el sumaria es a nivel de vivienda y no de persona, primero se debe unir los modulos 2, 3 y 5

In [3]:
dta = set2.merge(set3, how='inner').merge(set4, how='inner')

Finalmente, haremos el merge con la sumaria:

In [4]:
dta = dta.merge(set1, how='inner')

### Modificaciones a las variables

In [5]:
columnas = list(dta.columns)

print(columnas)

['mes', 'ubigeo', 'conglome', 'vivienda', 'estrato', 'codperso', 'p208a', 'p207', 'p209', 'p301a', 'p301d', 'p512a', 'p514', 'ingmo1hd', 'pobreza']


Renombraremos las columnas

In [6]:
dta.rename(
    {"p208a": "edad", "p207": "sexo", "p209": "estado_civil", "p301a": "nivel_educ", "p301d": "centro_estudios",
     "p512a": "tamano_empres", "p514": "ocupacion_secund", "ingmo1hd": "ingbrut"}, axis=1, inplace=True)

dta.head()

,mes,ubigeo,conglome,vivienda,estrato,codperso,edad,sexo,estado_civil,nivel_educ,centro_estudios,tamano_empres,ocupacion_secund,ingbrut,pobreza
0,01,010101,005009,041,de 20 000 a 49 999 habitantes,01,60.0,mujer,soltero(a),sin nivel,NaN,hasta 20 personas,si,17454.0,no pobre
1,01,010101,005009,041,de 20 000 a 49 999 habitantes,02,28.0,mujer,soltero(a),superior universitaria completa,estatal,hasta 20 personas,si,17454.0,no pobre
2,01,010101,005009,056,de 20 000 a 49 999 habitantes,01,48.0,mujer,separado(a),secundaria incompleta,estatal,hasta 20 personas,si,24444.0,no pobre
3,01,010101,005009,056,de 20 000 a 49 999 habitantes,02,25.0,mujer,soltero(a),secundaria completa,estatal,hasta 20 personas,no,24444.0,no pobre
4,01,010101,005009,056,de 20 000 a 49 999 habitantes,03,16.0,mujer,soltero(a),secundaria completa,estatal,hasta 20 personas,no,24444.0,no pobre


La variable de `estrato` puede ser modificada de tal manera que las 5 primeras categorias representen el vivir en zona urbana y de la 6ta a la 8va que sea rural

In [7]:
categorias_urbana = list(dta["estrato"].cat.categories[:5])

dta["area"] = "rural"
dta.loc[dta["estrato"].isin(categorias_urbana), "area"] = "urbana"

Una vez importada, podemos eliminar las variables que ya no se usaran como: `mes`, `ubigeo`, `conglome`, `vivienda`, `codperso`

In [8]:
dta = dta.drop(["mes", "ubigeo", "conglome", "vivienda", "estrato", "codperso"], axis=1)
dta.head()

,edad,sexo,estado_civil,nivel_educ,centro_estudios,tamano_empres,ocupacion_secund,ingbrut,pobreza,area
0,60.0,mujer,soltero(a),sin nivel,NaN,hasta 20 personas,si,17454.0,no pobre,urbana
1,28.0,mujer,soltero(a),superior universitaria completa,estatal,hasta 20 personas,si,17454.0,no pobre,urbana
2,48.0,mujer,separado(a),secundaria incompleta,estatal,hasta 20 personas,si,24444.0,no pobre,urbana
3,25.0,mujer,soltero(a),secundaria completa,estatal,hasta 20 personas,no,24444.0,no pobre,urbana
4,16.0,mujer,soltero(a),secundaria completa,estatal,hasta 20 personas,no,24444.0,no pobre,urbana


Las variables que quedan por el momento son las siguientes:
* **edad**: edad
* **sexo**: dicotomica de hombre y mujer
* **estado_civil**: estado civil del encuestado
* **nivel_educ**: mayor nivel educativo aprobado
* **centro_estudios**: colegio de procedencia: estatal o no estatal
* **tamano_empres**: cantidad de trabajadores dentro de la empresa que trabaje
* **ocupacion_secud**: dicotomica si cuenta con un segundo trabajo o no
* **ingbrut**: ingresos monetarios brutos
* **pobreza**: dicotomica si es pobre o no
* **area**: categorias donde están los que viven en zona urbana o rural

### Revisión de las variables

Primero, se puede observar que los tipos de las variables son los siguientes:

In [9]:
print(f"La cantidad de observaciones son {dta.shape[0]}, y la cantidad de variables son: {dta.shape[1]}")

La cantidad de observaciones son 115159, y la cantidad de variables son: 10


In [10]:
dta.dtypes

edad                 float64
sexo                category
estado_civil        category
nivel_educ          category
centro_estudios     category
tamano_empres       category
ocupacion_secund    category
ingbrut              float64
pobreza             category
area                  object
dtype: object

In [11]:
dta.describe()

,edad,ingbrut
count,115133.000000,115159.000000
mean,41.963607,32301.096864
std,19.537867,32338.833127
min,0.000000,0.000000
25%,25.000000,11460.000000
50%,40.000000,23338.000000
75%,57.000000,41802.000000
max,98.000000,540364.000000


Los _missing values_ (True) de las variables son las siguentes:

In [12]:
missing_data = dta.isnull()

for i in missing_data.columns.values.tolist():
    print(f"{missing_data[i].value_counts()}\n")

False    115133
True         26
Name: edad, dtype: int64

False    115133
True         26
Name: sexo, dtype: int64

False    113898
True       1261
Name: estado_civil, dtype: int64

False    115118
True         41
Name: nivel_educ, dtype: int64

False    108837
True       6322
Name: centro_estudios, dtype: int64

False    85570
True     29589
Name: tamano_empres, dtype: int64

False    86057
True     29102
Name: ocupacion_secund, dtype: int64

False    115159
Name: ingbrut, dtype: int64

False    115159
Name: pobreza, dtype: int64

False    115159
Name: area, dtype: int64



Si se elimina las observaciones que tienen _missing values_ para `tamano_empres`, `centro_estudios` y `estado_civil`, nos quedamos sin _missing values_.

In [13]:
dta = dta.dropna(subset=["tamano_empres", "centro_estudios", "estado_civil"], axis=0)

In [14]:
missing_data = dta.isnull()

for i in missing_data.columns.values.tolist():
    print(f"{missing_data[i].value_counts()}\n")

False    80419
Name: edad, dtype: int64

False    80419
Name: sexo, dtype: int64

False    80419
Name: estado_civil, dtype: int64

False    80419
Name: nivel_educ, dtype: int64

False    80419
Name: centro_estudios, dtype: int64

False    80419
Name: tamano_empres, dtype: int64

False    80419
Name: ocupacion_secund, dtype: int64

False    80419
Name: ingbrut, dtype: int64

False    80419
Name: pobreza, dtype: int64

False    80419
Name: area, dtype: int64



Ahora haré una breve exploracion de las variables categoricas:

In [15]:
dta["sexo"].value_counts()

hombre    43214
mujer     37205
Name: sexo, dtype: int64

In [16]:
dta["estado_civil"].value_counts()

conviviente      24890
casado(a)        22920
soltero(a)       20674
separado(a)       8621
viudo(a)          2911
divorciado(a)      403
Name: estado_civil, dtype: int64

In [17]:
dta["nivel_educ"].value_counts()

secundaria completa                     21196
secundaria incompleta                   13658
primaria incompleta                     11615
primaria completa                       10423
superior no universitaria completa       7787
superior universitaria completa          6416
superior universitaria incompleta        4157
superior no universitaria incompleta     3581
maestria/doctorado                       1450
educación inicial                         111
básica especial                            25
sin nivel                                   0
Name: nivel_educ, dtype: int64

In [18]:
dta["centro_estudios"].value_counts()

estatal       65868
no estatal    14551
Name: centro_estudios, dtype: int64

In [19]:
dta["tamano_empres"].value_counts()

hasta 20 personas        63369
más de 500 personas      10444
de 101 a 500 personas     2785
de 21 a 50 personas       2261
de 51 a 100 personas      1560
Name: tamano_empres, dtype: int64

In [20]:
dta["ocupacion_secund"].value_counts()

no    63919
si    16500
Name: ocupacion_secund, dtype: int64

In [21]:
dta["pobreza"].value_counts()

no pobre            67071
pobre no extremo    11437
pobre extremo        1911
Name: pobreza, dtype: int64

In [22]:
dta["area"].value_counts()

urbana    54838
rural     25581
Name: area, dtype: int64

### Reemplazo de variables

En esta nueva seccion, creare nuevas variables para el codigo de la estimacion de los modelos

In [23]:
# Edad
dta_f = dta[["edad"]].copy()

# Hombre
dta_f["hombre"] = pd.get_dummies(dta["sexo"])["hombre"].astype("float")

# Estado civil
# dta_f["estado_civil"] = dta["estado_civil"].copy()

# Estudios superiores
estudios_superiores = ["superior no universitaria completa", "superior universitaria completa", "maestria/doctorado"]
dta_f["est_sup"] = 0.0
dta_f.loc[dta["nivel_educ"].isin(estudios_superiores), "est_sup"] = 1.0

# Colegio privado
dta_f["colegio_priv"] = pd.get_dummies(dta["centro_estudios"])["no estatal"].astype("float")

# Trabajar en gran empresa
gran_empresa = ["de 101 a 500 personas", "más de 500 personas"]
dta_f["gran_empresa"] = 0.0
dta_f.loc[dta["tamano_empres"].isin(gran_empresa), "gran_empresa"] = 1.0

# Tener segundo trabajo
dta_f["segundo_trab"] = pd.get_dummies(dta["ocupacion_secund"])["si"].astype("float")

# Ingresos brutos
dta_f["ingbrut"] = dta[["ingbrut"]].copy()/12

# Vivir en zona urbana
dta_f["urbano"] = pd.get_dummies(dta["area"])["urbana"].astype("float")

# Pobreza
pobreza = ["pobre extremo", "pobre no extremo"]
dta_f["no_pobre"] = 1.0
dta_f.loc[dta["pobreza"].isin(pobreza), "no_pobre"] = 0.0

In [24]:
dta_f.head()

,edad,hombre,est_sup,colegio_priv,gran_empresa,segundo_trab,ingbrut,urbano,no_pobre
1,28.0,0.0,1.0,0.0,0.0,1.0,1454.500000,1.0,1.0
2,48.0,0.0,0.0,0.0,0.0,1.0,2037.000000,1.0,1.0
3,25.0,0.0,0.0,0.0,0.0,0.0,2037.000000,1.0,1.0
4,16.0,0.0,0.0,0.0,0.0,0.0,2037.000000,1.0,1.0
6,41.0,0.0,0.0,0.0,0.0,1.0,1842.066732,1.0,1.0


In [25]:
dta_f["no_pobre"].value_counts()

1.0    67071
0.0    13348
Name: no_pobre, dtype: int64

### Guardado

Primero se elimina todos los archivos que se encontraban en la carpeta de base de datos

In [26]:
for file in os.listdir("Bases de datos"):
    os.remove(f"Bases de datos//{file}")

Finalmente, se guarda los datos del DataFrame `dta_f`

In [27]:
dta_f.to_csv("./Bases de datos/data_f.csv", index_label=None)